In [1]:
import os
import h5py
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold,StratifiedKFold, GroupKFold, KFold
import nilearn as nl
import torch
import random
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
import itertools

import monai
from monai.transforms import \
    LoadNifti, LoadNiftid, AddChanneld, ScaleIntensityRanged, \
    Rand3DElasticd, RandAffined, \
    Spacingd, Orientationd

root = r'/home/youhanlee/project/trends/input/'
#root = r'/home/iclab/projects/Trends/input/'

#train = pd.read_csv('{}/train_scores.csv'.format(root)).sort_values(by='Id')
loadings = pd.read_csv('{}/loading.csv'.format(root))
sample = pd.read_csv('{}/sample_submission.csv'.format(root))
reveal = pd.read_csv('{}/reveal_ID_site2.csv'.format(root))
ICN = pd.read_csv('{}/ICN_numbers.csv'.format(root))
fnc = pd.read_csv('{}/fnc.csv'.format(root))



"""
    Load and display a subject's spatial map
"""

def load_subject(filename, mask_niimg):
    """
    Load a subject saved in .mat format with the version 7.3 flag. Return the subject niimg, using a mask niimg as a template for nifti headers.
    Args:
        filename    <str>            the .mat filename for the subject data
        mask_niimg  niimg object     the mask niimg object used for nifti headers
    """
    subject_data = None
    with h5py.File(filename, 'r') as f:
        subject_data = f['SM_feature'][()]
        # print(subject_data.shape)
    # It's necessary to reorient the axes, since h5py flips axis order
    subject_data = np.moveaxis(subject_data, [0, 1, 2, 3], [3, 2, 1, 0])
    # print(subject_data.shape)
    return subject_data
    # subject_niimg = nl.image.new_img_like(mask_niimg, subject_data, affine=mask_niimg.affine, copy_header=True)
    # return subject_niimg

def read_data_sample():
    # Input data files are available in the "../input/" directory.
    # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
    mask_filename = r'{}/fMRI_mask.nii'.format(root)
    subject_filename = '{}/fMRI_train/10004.mat'.format(root)

    mask_niimg = nl.image.load_img(mask_filename)
    print("mask shape is %s" % (str(mask_niimg.shape)))

    subject_niimg = load_subject(subject_filename, mask_niimg)
    print("Image shape is %s" % (str(subject_niimg.shape)))
    num_components = subject_niimg.shape[-1]
    print("Detected {num_components} spatial maps".format(num_components=num_components))

class TReNDsDataset(Dataset):

    def __init__(self, mode='train', fold_index = 0, feat_index=0):
        # print("Processing {} datas".format(len(self.img_list)))
        self.mode = mode
        self.fold_index = fold_index
        self.feat_index = feat_index
        fnc = pd.read_csv('{}/fnc.csv'.format(root))
        fnc_columns = fnc.columns[1:]
        fnc[fnc_columns] /= 500
        
        degree = pd.read_csv('{}/degree.csv'.format(root))
        degree_columns = degree.columns[:-1]
        degree[degree_columns] /= 10000
        
        train = pd.read_csv('{}/train_scores.csv'.format(root)).sort_values(by='Id')
        
        if self.mode=='train' or self.mode=='valid' or self.mode=='valid_tta':
            features = ('age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2')
            good_feats = [f'IC_{feat}' for feat in ['15', '22', '06', '21', '04', '11', '10', '16']]
            train = train[['Id', features[self.feat_index]]]
            print(train.columns)
            
            data = pd.merge(loadings, train, on='Id').dropna()
            comb = itertools.combinations(good_feats, 2)
            for col1, col2 in comb:
                data[f'{col1}_x_{col2}'] = data[col1] * data[col2]
                data[f'{col1}_+_{col2}'] = data[col1] + data[col2]
            
            id_train = list(data.Id)
            fea_train = np.asarray(data.drop([features[self.feat_index]], axis=1).drop('Id', axis=1))
            lbl_train = np.asarray(data[[features[self.feat_index]]])
            
            # fnc
            data = pd.merge(fnc, train, on='Id').dropna()
            fnc_train = np.asarray(data.drop([features[self.feat_index]], axis=1).drop('Id', axis=1))
            
            # degree
            data = pd.merge(degree, train, on='Id').dropna()
            deg_train = np.asarray(data.drop([features[self.feat_index]], axis=1).drop('Id', axis=1))

            self.all_samples = []
            for i in tqdm(range(len(id_train))):
                id = id_train[i]
                fea = fea_train[i]
                fnc = fnc_train[i]
                deg = deg_train[i]
                lbl = lbl_train[i]
                filename = os.path.join('{}/fMRI_train_npy/{}.npy'.format(root, id))
                self.all_samples.append([filename, fea, fnc, deg, lbl, str(id)])
                
            fold_df = data[[features[self.feat_index]]]
            min_value = fold_df[f'{features[self.feat_index]}'].min()
            max_value = fold_df[f'{features[self.feat_index]}'].max()
            binsplits = np.arange(min_value,max_value,15)
            y_categorized = np.digitize(fold_df[f'{features[self.feat_index]}'].values, bins=binsplits)
            
            fold_df[f'{features[self.feat_index]}_cat'] = y_categorized

            train_indexes = []
            valid_indexes = []
            kf = StratifiedShuffleSplit(n_splits=5,  random_state=42)
            for trn_idx, vld_idx in kf.split(fold_df, fold_df[f'{features[self.feat_index]}_cat']):
                train_indexes.append(trn_idx)
                valid_indexes.append(vld_idx)
                
            self.train_index = train_indexes[fold_index]
            self.valid_index = valid_indexes[fold_index]

            if self.mode=='train':
                self.train_index = [tmp for tmp in self.train_index if os.path.exists(self.all_samples[tmp][0])]
                self.len = len(self.train_index)
                print('fold index:',fold_index)
                print('train num:', self.len)

            elif self.mode=='valid' or self.mode=='valid_tta':
                self.valid_index = [tmp for tmp in self.valid_index if os.path.exists(self.all_samples[tmp][0])]
                self.len = len(self.valid_index)
                print('fold index:',fold_index)
                print('valid num:', self.len)

        elif  self.mode=='test':
            labels_df = pd.read_csv("{}/train_scores.csv".format(root))
            labels_df["is_train"] = True

            features = ('age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2')
            labels_df = labels_df[['Id', 'is_train', features[self.feat_index]]]
            print(labels_df.columns)
            data = pd.merge(loadings, labels_df, on="Id", how="left")
            
            good_feats = [f'IC_{feat}' for feat in ['15', '22', '06', '21', '04', '11', '10', '16']]
            #data = pd.merge(loadings, train, on='Id').dropna()
            comb = itertools.combinations(good_feats, 2)
            for col1, col2 in comb:
                data[f'{col1}_x_{col2}'] = data[col1] * data[col2]
                data[f'{col1}_+_{col2}'] = data[col1] + data[col2]

            id_test = list(data[data["is_train"] != True].Id)
            fea_test = np.asarray(data.drop([features[self.feat_index]], axis=1).drop('Id', axis=1)[data["is_train"] != True].drop("is_train", axis=1))
            lbl_test = np.asarray(data[[features[self.feat_index]]][data["is_train"] != True])
            
            data = pd.merge(fnc, labels_df, on="Id", how="left")
            fnc_test = np.asarray(data.drop([features[self.feat_index]], axis=1).drop('Id', axis=1)[data["is_train"] != True].drop("is_train", axis=1))
            
            # degree
            data = pd.merge(degree, labels_df, on='Id', how='left')
            deg_test = np.asarray(data.drop([features[self.feat_index]], axis=1).drop('Id', axis=1)[data["is_train"] != True].drop("is_train", axis=1))
            #print(deg_test.shape)

            self.all_samples = []
            for i in range(len(id_test)):
                id = id_test[i]
                fea = fea_test[i]
                fnc = fnc_test[i]
                deg = deg_test[i]
                lbl = lbl_test[i]

                filename = os.path.join('{}/fMRI_test_npy/{}.npy'.format(root, id))
                if os.path.exists(filename):
                    self.all_samples.append([id, filename, fea, fnc, deg, lbl])
                    #all_samples.append([filename, fea, fnc, lbl, str(id)])

            self.len = len(self.all_samples)
            print(len(id_test))
            print('test num:', self.len)

    def __getitem__(self, idx):

        if self.mode == "train" :
            filename, feat, fnc, deg, lbl, id =  self.all_samples[self.train_index[idx]]
            train_img = np.load(filename).astype(np.float32)
            train_img = train_img.transpose((3,2,1,0))
            # (53, 52, 63, 53)
            train_lbl = lbl

            data_dict = {'image': train_img}
            rand_affine = RandAffined(keys=['image'],
                                      mode=('bilinear', 'nearest'),
                                      prob=0.5,
                                      spatial_size=(52, 63, 53),
                                      translate_range=(5, 5, 5),
                                      rotate_range=(np.pi * 4, np.pi * 4, np.pi * 4),
                                      scale_range=(0.15, 0.15, 0.15),
                                      padding_mode='border')
            affined_data_dict = rand_affine(data_dict)
            train_img = affined_data_dict['image']
            train_feat = feat #affined_data_dict['feat']
            train_fnc = fnc#affined_data_dict['fnc']
            train_deg = deg#affined_data_dict['deg']

            return torch.FloatTensor(train_img), \
                   torch.FloatTensor(train_feat), \
                   torch.FloatTensor(train_fnc), \
                   torch.FloatTensor(train_deg), \
                   torch.FloatTensor(train_lbl)

        elif self.mode == "valid":
            filename, feat, fnc, deg, lbl, id =  self.all_samples[self.valid_index[idx]]
            train_img = np.load(filename).astype(np.float32)
            train_img = train_img.transpose((3, 2, 1, 0))
            # (53, 52, 63, 53)
            train_lbl = lbl

            return torch.FloatTensor(train_img), \
                   torch.FloatTensor(feat), \
                   torch.FloatTensor(fnc), \
                   torch.FloatTensor(deg), \
                   torch.FloatTensor(train_lbl)


        elif self.mode == 'test':
            id, filename, feat, fnc, deg, lbl =  self.all_samples[idx]
            test_img = np.load(filename).astype(np.float32)
            test_img = test_img.transpose((3, 2, 1, 0))
            
            return str(id), \
                   torch.FloatTensor(test_img), \
                   torch.FloatTensor(feat), \
                   torch.FloatTensor(fnc), \
                   torch.FloatTensor(deg)

    def __len__(self):
        return self.len


In [2]:
dataset = TReNDsDataset(mode='valid', fold_index=0, feat_index=4)

Index(['Id', 'domain2_var2'], dtype='object')


100%|██████████| 5838/5838 [00:00<00:00, 423741.85it/s]

fold index: 0
valid num: 584



/home/youhanlee/anaconda3/envs/lyh/lib/python3.7/site-packages/ipykernel_launcher.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [55]:
train_loader = torch.utils.data.DataLoader(dataset, batch_size=4)

In [56]:
batch = next(iter(train_loader))

In [59]:
batch[4].shape

torch.Size([4, 1])

In [3]:
foo = dataset[0]

In [4]:
foo[0].shape

torch.Size([53, 52, 63, 53])

In [5]:
foo[1].shape

torch.Size([82])

In [6]:
foo[2].shape

torch.Size([1378])

In [7]:
foo[3].shape

torch.Size([53])

In [8]:
foo[4].shape

torch.Size([1])

In [9]:
foo[4]

tensor([39.6290])

In [10]:
import torch

In [47]:
loss_fn = torch.nn.SmoothL1Loss()

In [48]:
outputs = torch.rand(16, 1)

In [49]:
labels = torch.rand(16, 1)

In [50]:
loss_fn(outputs, labels)

tensor(0.0594)

In [39]:
def nae(inp, targ):
    return torch.mean(torch.abs(inp - targ)) / torch.mean(targ)

In [45]:
torch.mean(torch.abs(outputs - labels)) / torch.mean(labels)

tensor(0.5592)

In [46]:
torch.mean(torch.abs(outputs.squeeze() - labels)) / torch.mean(labels)

tensor(0.5456)

In [35]:
torch.mean(torch.abs(outputs - labels) / torch.mean(labels))

tensor(0.5843)

In [36]:
torch.abs(outputs - labels)

tensor([[1.4094e-02, 3.7942e-01, 8.1290e-01, 2.9592e-01, 8.0429e-01, 2.2818e-01,
         8.0552e-01, 1.7403e-01, 1.4870e-01, 3.4781e-01, 1.3194e-01, 2.1497e-01,
         3.4613e-01, 4.9025e-01, 1.0067e-01, 4.5689e-01],
        [3.4789e-01, 1.7436e-02, 4.5092e-01, 6.6056e-02, 4.4231e-01, 1.3380e-01,
         4.4354e-01, 1.8795e-01, 5.1068e-01, 1.4167e-02, 2.3004e-01, 1.4701e-01,
         1.5849e-02, 1.2827e-01, 4.6265e-01, 9.4910e-02],
        [2.4063e-01, 1.2469e-01, 5.5818e-01, 4.1202e-02, 5.4957e-01, 2.6541e-02,
         5.5080e-01, 8.0693e-02, 4.0342e-01, 9.3091e-02, 1.2278e-01, 3.9747e-02,
         9.1410e-02, 2.3553e-01, 3.5540e-01, 2.0217e-01],
        [4.8157e-01, 1.1625e-01, 3.1723e-01, 1.9974e-01, 3.0863e-01, 2.6749e-01,
         3.0985e-01, 3.2164e-01, 6.4437e-01, 1.4785e-01, 3.6373e-01, 2.8069e-01,
         1.4953e-01, 5.4163e-03, 5.9634e-01, 3.8776e-02],
        [7.4709e-02, 2.9061e-01, 7.2409e-01, 2.0712e-01, 7.1549e-01, 1.3938e-01,
         7.1671e-01, 8.5224e-02, 2.3751

In [30]:
torch.mean(torch.abs(outputs - labels) / labels)

tensor(0.3837)